In [1]:
import spacy
import random
import json

In [34]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def generate_better_characters(file):
    characters = load_data(file)
    new_characters = [character for character in characters]
    print("Original number of new_characters {}".format(len(new_characters)))
    
    for item in characters:
        item  = item.replace("The", "").replace("the", "").replace("and", "").replace("And", "")
        names = item.split(" ")
        for name in names:
            name = name.strip()
            new_characters.append(name)
        if "(" in item:
            names = item.split("(")
            for name in names:
                name = name.replace(")", "").strip()
                new_characters.append(name)
        if "," in item:
            names = item.split(",")
            for name in names:
                name = name.replace("and", "").strip()
                if " " in name:
                    new_names = name.split()
                    for x in new_names:
                        x = x.strip()
                        new_characters.append(x)
                new_characters.append(name)
    final_characters = []
    titles = ["Dr.", "Professor", "Mr.", "Mrs.", "Ms.", "Miss", "Aunt", "Uncle", "Mr. and Mrs."]
    
    for character in new_characters:
        if "" != character:
            final_characters.append(character)
            for title in titles:
                titled_char = f"{title} {character}"
                final_characters.append(titled_char)
            
    final_characters = list(set(final_characters))
    final_characters.sort()
    
    print("After stripping, total number of new characters {}".format(len(final_characters)))
    return final_characters


def create_training_data(file, label_type):
    data = generate_better_characters(file)
    patterns = []
    for item in data:
        pattern = {
            "label": label_type,
            "pattern": item
        }
        patterns.append(pattern)
    return patterns

def generate_rules(patterns):
    nlp = English()
    ruler = EntityRuler(nlp)
    ruler.add_patterns(patterns)
    nlp.add_pipe(ruler)
    nlp.to_disk('hr_ner')

def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        single_training_data = (ent.start_char, ent.end_char, ent.label_)
        entities.append((ent.start_char, ent.end_char, ent.label_))
    
    if len(entities) > 0:
        results = (text, {"entities":entities})

    return results

def save_data(file, data):
    with open(file, "w", encoding='utf-8') as file:
        json.dump(data, file, indent=4)
    

In [37]:
nlp = spacy.load("hr_ner")
TRAIN_DATA = []

with open("../data/digital_humanity/hp.txt") as f:
    text = f.read()
    chapters = text.split("CHAPTER")[1:]
    ie_data = {}
    for chapter in chapters:
        chapter_num, chapter_title =  chapter.split("\n\n")[0:2]
        chapter_num = chapter_num.strip()
        segments = chapter.split("\n\n")[2:]
        hits = []
        
        for segment in segments:
            segment = segment.strip()
            segment = segment.replace("\n", " ")
            results =  test_model(nlp,segment)
            
            if len(results) > 0:
                TRAIN_DATA.append(results)
save_data("../data/hp_training_data.json", TRAIN_DATA)


In [38]:
training_data = load_data("../data/hp_training_data.json")

list

In [43]:
len(TRAIN_DATA)

2213

123